In [15]:
from config.spark import SparkIcebergConfig

# Stop existing session
if 'spark' in globals():
    spark.stop()

# Create new session with desired configuration
spark = SparkIcebergConfig.create_session("glue")  # or "glue" or "rest"

In [16]:
%%sql
SHOW NAMESPACES IN glue;

25/09/15 09:46:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace
demo


In [3]:
%%sql 
CREATE NAMESPACE IF NOT EXISTS glue.demo;

++
||
++
++

In [4]:
%%sql
SHOW NAMESPACES IN glue;

namespace
demo
`test-glue-database`


In [9]:
%%sql
CREATE TABLE
  IF NOT EXISTS glue.demo.users (id BIGINT, name STRING, age INT) USING ICEBERG PARTITIONED BY (age);

++
||
++
++

In [4]:
%%sql
INSERT INTO
  glue.demo.users
VALUES
  (1, 'Alice', 30),
  (2, 'Bob', 25),
  (3, 'Charlie', 35);

25/09/14 16:22:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [5]:
%%sql
SELECT
  *
FROM
  glue.demo.users;

id,name,age
2,Bob,25
1,Alice,30
3,Charlie,35
2,Bob,25
1,Alice,30
3,Charlie,35
1,Alice,30
3,Charlie,35


In [15]:
%%sql
DELETE FROM glue.demo.users
WHERE
  id = 2;

++
||
++
++

In [16]:
%%sql
SELECT
  *
FROM
  glue.demo.users;

id,name,age
1,Alice,30
3,Charlie,35


In [23]:
%%sql
DROP TABLE IF EXISTS glue.demo.users5;

++
||
++
++

In [25]:
%%sql
CREATE TABLE
  IF NOT EXISTS glue.demo.users5 (id4 BIGINT, name4 STRING, age4 INT) USING ICEBERG PARTITIONED BY (age4);

++
||
++
++

In [30]:
%%sql
INSERT INTO
  glue.demo.users4
VALUES
  (1, 'Alice', 23),
  (2, 'Bob', 34),
  (3, 'Charlie', 45);

++
||
++
++

In [7]:
%%sql
SELECT
  *
FROM
  glue.demo.users4.files;

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,referenced_data_file,content_offset,content_size_in_bytes,readable_metrics
0,s3://9565atulverma-s3-bucket/demo.db/users4/data/age4=23/00000-11-c505828a-0943-4589-baf3-674c2dda0e67-0-00001.parquet,PARQUET,0,Row(age4=23),1,931,"{1: 43, 2: 44, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Alice'), 3: bytearray(b'\x17\x00\x00\x00')}","{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Alice'), 3: bytearray(b'\x17\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(age4=Row(column_size=39, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=23, upper_bound=23), id4=Row(column_size=43, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=1), name4=Row(column_size=44, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='Alice', upper_bound='Alice'))"
0,s3://9565atulverma-s3-bucket/demo.db/users4/data/age4=45/00000-11-c505828a-0943-4589-baf3-674c2dda0e67-0-00003.parquet,PARQUET,0,Row(age4=45),1,945,"{1: 43, 2: 46, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: bytearray(b'\x03\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Charlie'), 3: bytearray(b'-\x00\x00\x00')}","{1: bytearray(b'\x03\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Charlie'), 3: bytearray(b'-\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(age4=Row(column_size=39, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=45, upper_bound=45), id4=Row(column_size=43, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=3, upper_bound=3), name4=Row(column_size=46, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='Charlie', upper_bound='Charlie'))"
0,s3://9565atulverma-s3-bucket/demo.db/users4/data/age4=34/00000-11-c505828a-0943-4589-baf3-674c2dda0e67-0-00002.parquet,PARQUET,0,Row(age4=34),1,917,"{1: 43, 2: 42, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Bob'), 3: bytearray(b'""\x00\x00\x00')}","{1: bytearray(b'\x02\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Bob'), 3: bytearray(b'""\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(age4=Row(column_size=39, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=34, upper_bound=34), id4=Row(column_size=43, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=2, upper_bound=2), name4=Row(column_size=42, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='Bob', upper_bound='Bob'))"
0,s3://9565atulverma-s3-bucket/demo.db/users4/data/age4=23/00000-3-19c2b588-4d84-413c-870b-a854313c7443-0-00001.parquet,PARQUET,0,Row(age4=23),1,931,"{1: 43, 2: 44, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Alice'), 3: bytearray(b'\x17\x00\x00\x00')}","{1: bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Alice'), 3: bytearray(b'\x17\x00\x00\x00')}",None,[4],None,0,None,None,None,"Row(age4=Row(column_size=39, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=23, upper_bound=23), id4=Row(column_size=43, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=1, upper_bound=1), name4=Row(column_size=44, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='Alice', upper_bound='Alice'))"
0,s3://9565atulverma-s3-bucket/demo.db/users4/data/age4=45/00000-3-19c2b588-4d84-413c-870b-a854313c7443-0-00003.parquet,PARQUET,0,Row(age4=45),1,945,"{1: 43, 2: 46, 3: 39}","{1: 1, 2: 1, 3: 1}","{1: 0, 2: 0, 3: 0}",{},"{1: bytearray(b'\x03\x00\x00\x00\x00\x00\x00\x00'), 2: bytearray(b'Charlie'), 3: bytearray(b'-\x00\x00\x00')}","{1: bytearray(b'\x03\x00\x00\x00\x00\x00\x00

In [23]:
### Time Travel

In [17]:
%%sql
SELECT
  *
FROM
  glue.demo.users4 TIMESTAMP AS OF '2025-09-13 23:21:00';

id4,name4,age4
1,Alice,23
3,Charlie,45
2,Bob,34


In [18]:
%%sql
SELECT
  *
FROM
  glue.demo.users4 TIMESTAMP AS OF '2025-09-14 19:54:00';

id4,name4,age4
1,Alice,23
3,Charlie,45
2,Bob,34
1,Alice,23
3,Charlie,45
2,Bob,34
1,Alice,23
3,Charlie,45
2,Bob,34


In [32]:
%%sql
SELECT
  id4,	name4
FROM
  glue.demo.users4;

id4,name4
1,Alice
3,Charlie
2,Bob
1,23
2,34
3,45
1,Alice
3,Charlie
2,Bob
1,Alice


In [35]:
%%sql
select * from glue.demo.users4;

id4,name4,age4
1,Alice,23
3,Charlie,45
2,Bob,34
1,Alice,23
3,Charlie,45
2,Bob,34
1,Alice,23
3,Charlie,45
2,Bob,34
1,23,None


In [12]:
%%sql
INSERT INTO
  glue.demo.users4(id4, age4, name4)
VALUES
  (1, 23, 'a'),
  (2, 34, 'a'),
  (3, 45, 'a');

++
||
++
++

In [37]:
%%sql
TRUNCATE table glue.demo.users4;


++
||
++
++

In [18]:
%%sql
select * from glue.demo.users4;

id4,name4,age4
1,a,23
3,a,45
2,a,34
1,a,23
3,a,45
2,a,34
1,a,23
3,a,45
2,a,34


In [19]:
%%sql
CREATE TABLE
  IF NOT EXISTS glue.demo.users6 (id BIGINT, name STRING, age INT) USING ICEBERG PARTITIONED BY (age);

Py4JJavaError: An error occurred while calling o390.sql.
: software.amazon.awssdk.services.glue.model.AccessDeniedException: Insufficient Lake Formation permission(s) on users6 (Service: Glue, Status Code: 400, Request ID: 2e475a49-7794-45d5-96b8-356a124dbced)
	at software.amazon.awssdk.core.internal.http.CombinedResponseHandler.handleErrorResponse(CombinedResponseHandler.java:124)
	at software.amazon.awssdk.core.internal.http.CombinedResponseHandler.handleResponse(CombinedResponseHandler.java:81)
	at software.amazon.awssdk.core.internal.http.CombinedResponseHandler.handle(CombinedResponseHandler.java:59)
	at software.amazon.awssdk.core.internal.http.CombinedResponseHandler.handle(CombinedResponseHandler.java:40)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.HandleResponseStage.execute(HandleResponseStage.java:50)
	at software.amazon.awssdk.core.internal.http.pipeline.stages.HandleResponseStage.execute(HandleResponseStage.java:38)
	at software.amazon.awssdk.core.interna